# Develop a simple regression model to be used in the web service

### Imports

In [25]:
import __main__ as main

from helpers.paths import Paths
from joblib import load, dump
from helpers.is_interactive import is_interactive
from functools import reduce

### Run dependency notebooks

In [26]:
if is_interactive(main):
    %run 01_0_data_wrangling.ipynb -p
    %run 02_0_scaling.ipynb -p

Running previous notebooks...


### Load Dataframe

In [27]:
data = load(Paths.REGRESSOR_DATA_WRANGLING_DATA)

df = data['dataset']

df = df.drop('price_cleaned', axis=1)

df.head(10)

,Living space,Plot area,Floor,Latitude,Longitude,Zip,distanceToTrainStation,gde_area_agriculture_percentage,gde_area_forest_percentage,gde_area_nonproductive_percentage,...,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa,NoisePollutionRailway,NoisePollutionRoad,PopulationDensity,RiversAndLakes,WorkplaceDensity,ForestDensity
0,100.0,1282.323307,4.000000,47.415927,8.085840,5023.0,3.038467,30.676329,51.449275,4.589372,...,0.0,0.0,0.0,0.0,0.000000,0.076399,0.223039,0.031951,0.060414,0.296178
1,156.0,222.000000,2.750000,47.415927,8.085840,5023.0,3.038467,30.676329,51.449275,4.589372,...,0.0,0.0,1.0,0.0,0.000000,0.076399,0.223039,0.031951,0.060414,0.296178
2,93.0,1198.982216,2.000000,47.397416,8.043150,5000.0,0.909587,11.354420,32.197891,7.137064,...,0.0,0.0,0.0,0.0,0.000000,0.337930,0.451622,0.114168,0.167442,0.087050
3,154.0,370.000000,0.000000,47.415927,8.085840,5023.0,3.038467,30.676329,51.449275,4.589372,...,0.0,0.0,0.0,0.0,0.000000,0.076399,0.223039,0.031951,0.060414,0.296178
4,142.0,462.537377,0.000000,47.404870,8.052781,5022.0,1.460245,33.137090,49.705635,1.177460,...,0.0,0.0,0.0,0.0,0.000000,0.167450,0.177506,0.114288,0.046822,0.252992
5,190.0,1063.000000,0.000000,47.401163,8.012034,5018.0,3.147978,32.994924,53.197970,1.218274,...,0.0,0.0,0.0,0.0,0.000000,0.158337,0.317534,0.000000,0.068993,0.129393
6,124.0,200.000000,2.750000,47.415927,8.085840,5023.0,3.038467,30.676329,51.449275,4.589372,...,0.0,0.0,1.0,0.0,0.000000,0.076399,0.223039,0.031951,0.060414,0.296178
7,112.0,94.942325,3.000000,47.388821,8.042194,5000.0,0.738825,11.354420,32.197891,7.137064,...,0.0,0.0,0.0,0.0,0.052450,0.387507,0.267420,0.000084,0.303656,0.017111
8,75.0,41.432719,1.620893,47.400929,8.070691,5032.0,1.018878,11.354420,32.197891,7.137064,...,0.0,0.0,0.0,0.0,0.008914,0.370294,0.313341,0.036454,0.090869,0.347674
9,110.0,2116.000000,0.000000,47.395295,8.012752,5018.0,2.818486,32.994924,53.197970,1.218274,...,1.0,0.0,0.0,0.0,0.000000,0.191687,0.229131,0.000000,0.067679,0.145702


## Prepare the metadata for the simple model

In [28]:
asked_features = {
    'Living space': {
        'label': 'Living Space',
        'value': 100.
    },
    'Plot area': {
        'label': 'Plot Area',
        'value': 100.    
    },
    'Zip': {
        'label': 'Zip Code'
    },
    'rooms': {
        'label': 'Number of rooms',
        'value': 4.5,
    },
    'type_unified': {
        'label': 'Type of property'
    },
    'Floor': {
        'label': 'Floor',
        'value': 1.
    }
}

## Find mean values per zip code to complete the data needed for the model

In [29]:
feature_means = reduce(
    lambda state, zip_code: {
        **state,
        zip_code: reduce(
            lambda state, col: {**state, col: df[df.Zip == zip_code][col].mean()},
            list(set([
                feat for feat in df.columns
                if feat not in asked_features
                and not feat.startswith('type_unified_')
            ])),
            {})
        },
    df.Zip.astype('int').unique(),
    {})

## Find the possible options for Zip code and Type

In [30]:
options = {
    'Zip': sorted(df.Zip.astype('int').unique()),
    'type_unified': [
        col.split('_')[-1]
        for col in df.columns
        if col.startswith('type_unified_')
    ]
}

In [31]:
scaling_data = load(Paths.REGRESSOR_SCALING_DATA)

In [32]:
dump({
    'asked_features': asked_features,
    'options': options,
    'feature_means': feature_means,
    'power_options': range(2, 7),
    'scaler': scaling_data['scaler']
}, Paths.WEBSERVICE_META_DATA)

['data/web_service/01_0_meta_data.dump']